In [870]:
import jieba
import numpy as np
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [871]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'dataTrainComplete'
#t_dir = 'dataPrivateComplete\dataPrivateComplete'
#t_dir = 'Stage_2\dataPublicComplete_s2\dataPublicComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))] #Article filenames

In [872]:
location_dict = {'台北':0,'台中':1,'台南':2,'苗栗':3,'彰化':4,'南投':5,'雲林':6,'嘉義':7,'屏東':8,'宜蘭':9,'花蓮':10,'台東':11}
area_dict = {'北部':[0],'東北部':[0,9,10,11],'中北部':[0,1,3,4,5,6],'中南部':[1,2,3,4,5,6,7,8]}
for ld in location_dict:jieba.add_word(ld)
for ad in location_dict:jieba.add_word(ad)

In [863]:
crop_list = open('Keywords/02crop.list_del.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()
#Keywords split by lines, keyword with more than one entry will be on the same line

In [864]:
from itertools import chain
import csv
vector_dict = {}
#Keyword lookup with keyword as key and vector index as value
keyword_count = 0
#for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep)):
    l = line.split(',')
    keyword_count += 1
    for word in l:
        #Some line will have more than one entry, which should have the same vector index
        if(word == '' or word == ' '):continue
        jieba.add_word(word)#Each keyword is added to jieba
        vector_dict[word] = idx
        
secondary_dict = {}
secondary_count = 0
for idx,line in enumerate(chem_line_sep):
    l = line.split(',')
    secondary_count += 1
    for word in l:
        #Some line will have more than one entry, which should have the same vector index
        if(word == '' or word == ' '):continue
        jieba.add_word(word)#Each keyword is added to jieba
        secondary_dict[word] = idx
keyword_count

373

In [865]:
vectors = {}
loc_vectors = {}
sec_vectors = {}
#Vector table with filename as key and vector as value
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [False]*keyword_count
    sec_vectors[fname] = [False]*secondary_count
    loc_vectors[fname] = [False]*12
    #Initialize an 0 vector for each file
    for seg in seg_list:
        if(seg in vector_dict):
            vectors[fname][vector_dict[seg]] = True
            #Count keyword appearence - Bag of words
        if(seg in location_dict):
            loc_vectors[fname][location_dict[seg]] = True
        if(seg in area_dict):
            for idx in area_dict[seg]:
                loc_vectors[fname][idx] = True
        if(seg in secondary_dict):
            sec_vectors[fname][secondary_dict[seg]] = True
    """
    for keyword in vector_dict:
        if(keyword in content):
            vectors[fname][vector_dict[keyword]] = True
    """
for key in vectors:
    v = vectors[key]
    curr = 0
    for b in v:
        if(b):curr = (curr << 1) + 1
        else:curr = curr << 1
    vectors[key] = curr
    
for key in sec_vectors:
    v = sec_vectors[key]
    curr = 0
    for b in v:
        if(b):curr = (curr << 1) + 1
        else:curr = curr << 1
    sec_vectors[key] = curr
"""
for key in loc_vectors:
    v = loc_vectors[key]
    curr = 0
    for b in v:
        if(b):curr = (curr << 1) + 1
        else:curr = curr << 1
    loc_vectors[key] = curr
"""

'\nfor key in loc_vectors:\n    v = loc_vectors[key]\n    curr = 0\n    for b in v:\n        if(b):curr = (curr << 1) + 1\n        else:curr = curr << 1\n    loc_vectors[key] = curr\n'

In [866]:
def countSetBits(n):
    count = 0
    while (n):
        n &= (n-1)
        count+= 1
    return count

In [867]:
corr_list = open('TrainLabel.csv', "r",encoding='UTF-8-sig')
corr = corr_list.read()
corr_line_sep = corr.splitlines()
pos_labels = set() #faster lookup to filter out positive pairs, not used elsewhere
#All the associated article pairs given by train label
for line in corr_line_sep[1:]:
    l = line.split(',')
    pos_pair_list.append((l[0],l[1]))
    pos_labels.add((l[0],l[1]))

In [868]:
out = []
loc_tracking = False
saved_entries = 0
failed = 0
for Ref in txt_fnames:
    for Test in txt_fnames:
        if(Ref == Test):continue
        if(countSetBits(vectors[Test] ^ (vectors[Test] & vectors[Ref])) == 0 and vectors[Test] != 0 and vectors[Ref] != 0):
            if(abs(countSetBits(vectors[Ref]) - countSetBits(vectors[Test])) < 3):
                if(sec_vectors[Test] == 0):
                    out.append((Test,Ref))
                elif(sec_vectors[Test] ^ (sec_vectors[Test] & sec_vectors[Ref]) == 0):
                    out.append((Test,Ref))

In [873]:
count = 0
bads = []
for o in out:
    if(o in pos_labels):count+=1
    else:bads.append(o)
recall,precision = count/len(pos_labels),count/len(out)
print(count,len(out))
print('Recall:',recall,'Precision:',precision,'F1',2*(recall*precision)/(recall+precision))

964 1915
Recall: 0.6970354302241504 Precision: 0.5033942558746737 F1 0.5845967252880533


In [808]:
for key in vectors:
    if(vectors[key] == 0):
        print(key)

1170
1178
887
969


In [809]:
tmp = {}
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    t = set()
    ad = False
    for seg in seg_list:
        if(seg in vector_dict):
            t.add(seg)
        if(seg == '地區'):
            ad = True
    if(ad):
        tmp[fname] = [t,content]
for k in tmp:
    print(k,tmp[k][1])

In [794]:
for Test,Ref in bads[:10]:
    txt = open(t_dir+'/'+Test+'.txt', "r",encoding="utf-8")
    content = txt.read()
    print(Test,content)
    txt = open(t_dir+'/'+Ref+'.txt', "r",encoding="utf-8")
    content = txt.read()
    print(Ref,content)
    print("--------------------------")

1007 乍暖還寒，防檢局籲請農友加強防治稻熱病
農委會防檢局表示，全國各地水稻生長陸續進入分蘗期，因逢暖冬，且最近天氣乍暖還寒，溫度變化大，再加上濕度高，非常容易發生稻熱病。依據田間監測資料顯示，中南部局部地區已有零星發生，該局特別呼籲全國農友應做好肥培、田間管理並依病情配合適時施藥防治，以防範稻熱病發生，確保稻米產量與品質。防檢局說，稻熱病為一種真菌性病害，在攝氏22度至30度及高濕度環境下容易發生。一期作水稻葉稻熱病多發生在3月下旬至5月上旬，尤其是4月上、中旬，也就是清明節過後半個月內最常發生，農友應把握時機切實防治。尤其是施用氮肥過多或接近山區的稻田，一旦田間發生病徵時，應立即施藥，以避免病害擴散蔓延。該局指出，稻熱病發病初期會在葉面上形成褐色或暗綠色小斑點，當環境適合時，病斑會逐漸擴大成紡錘形，其周圍呈黃色，中間赤褐色，內部灰白色，當受害嚴重時葉片枯萎甚至全株乾枯呈稻燒狀。稻熱病發病週期僅4至5天，故極易在短時間內爆發病害，若未加以防範，可能造成大面積蔓延危害。防檢局進一步說明，雖然稻熱病是本土水稻栽培的老病害，但每年隨氣候差異仍會造成不同程度的危害，該局籲請農友應隨時觀察田間病害發生情形，在插秧後35至50天，田間如發現葉稻熱病病徵時，應立即施藥一次，7天後，如有繼續蔓延，則再施藥一次。對該病的防治，農友可逕上防檢局農藥資訊服務網（http://pesticide.baphiq.gov.tw/）查詢或參考農委會編印之植物保護手冊用藥，並應遵守藥劑使用規定。防檢局同時表示，農友如有水稻稻熱病或其他植物病蟲害的問題，可以就近前往農委會所屬試驗改良場所請求協助診斷與指導，也可以透過植物病蟲害診斷服務專線：0800-069-880，與各地的診斷服務站專家聯繫洽詢。


103 桃園市及新竹縣部分鄉鎮區已發生葉稻熱病，為避免病害迅速蔓延，請農友適時防治。桃園區農改場、防檢局及田邊好幫手關心您。
依據桃園區農改場研究人員巡迴調查水稻疫情結果，轄內部分地區(桃園市楊梅區及新竹縣新豐鄉、竹北市、芎林鄉、北埔鄉、峨眉鄉等)已發生葉稻熱病，且近期氣候符合發病條件，為避免病害迅速蔓延，請農友適時防治。水稻葉稻熱病為一期稻作之重要病害，發病初期於葉面上形成褐色或暗綠色小斑點；如環境適合，則擴大成紡錘形，而病斑周圍呈黃色，中間赤褐色，內部灰白色；嚴重時葉片枯萎甚至全株枯死。此病

In [874]:
import csv
if(True):
    print(len(out))
    with open('val_super_simple_public_s1.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(["Test"]+["Reference"])
        for row in out:  
            spamwriter.writerow(row)


1915


In [857]:
false_negs = []
for pl in pos_labels:
    if(pl not in out):false_negs.append(pl)
for Test,Ref in false_negs[:10]:
    txt = open(t_dir+'/'+Test+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    keywrds = []
    for seg in seg_list:
        if(seg in vector_dict):keywrds.append(seg)
    print(keywrds)
    txt = open(t_dir+'/'+Ref+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    keywrds = []
    for seg in seg_list:
        if(seg in vector_dict):keywrds.append(seg)
    print(keywrds)
    print("--------")

FileNotFoundError: [Errno 2] No such file or directory: 'dataPrivateComplete\\dataPrivateComplete/780.txt'

0.5845331616451631